# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.python.framework.ops import Tensor

from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average, Maximum
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam
from keras.utils import to_categorical

from typing import Tuple, List
import glob
import os

# Load and prepare dataset

The CIFAR10 dataset contains 60 000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them. 

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels = to_categorical(train_labels, num_classes=10)

# CNN model creation for ensamble method

In [3]:
def cifar10_cnn_model(model_input: Tensor, model_name: str) -> training.Model:
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name=model_name)
    
    return model

def make_cnn_model(train_data: np.ndarray, model_name: str):
	input_shape = train_data[0,:,:,:].shape # (32,32,3)
	model_input = Input(shape=input_shape)

	return cifar10_cnn_model(model_input, model_name)

## Test train small model

In [4]:
NUM_EPOCHS = 15

# Note: Requires 'weights' folder to exist in order to save weights

def compile_and_train(model: training.Model, num_epochs: int, x_train: np.ndarray, y_train: np.ndarray, class_weights: dict) -> Tuple [History, str]: 
    
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto')
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train, y=y_train, batch_size=32, 
                     epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2, class_weight=class_weights)
    weight_files = glob.glob(os.path.join(os.getcwd(), 'weights/*'))
    weight_file = max(weight_files, key=os.path.getctime) # most recent file
    return history, weight_file

def evaluate_error(model: training.Model, test_data: np.ndarray, labels: np.ndarray) -> np.float64:
    pred = model.predict(test_data, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, labels)) / labels.shape[0]   
 
    return error

best_weight = {}
    

In [5]:
# Instantiate and display model 
car_plane_classifier = make_cnn_model(train_images, 'car_and_plane')
bird_cat_classifier = make_cnn_model(train_images, 'bird_and_cat')
deer_dog_classifier = make_cnn_model(train_images, 'deer_and_dog')
frog_horse_classifier = make_cnn_model(train_images, 'frog_and_horse')
ship_truck_classifier = make_cnn_model(train_images, 'ship_and_truck')

car_plane_classifier.summary()

Model: "car_and_plane"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 192)       166080    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 192)     

## Training the individual models

In [ ]:
car_plane_class_weight = {
    0: 5,
    1: 5,
    2: 1,
    3: 1,
    4: 1,
    5: 1,
    6: 1,
    7: 1,
    8: 1,
    9: 1
}
history_1, car_plane_classifier_weights = compile_and_train(car_plane_classifier, NUM_EPOCHS, train_images, train_labels, car_plane_class_weight)
best_weight[0] = car_plane_classifier_weights



In [ ]:
bird_cat_class_weight = {
    0: 1,
    1: 1,
    2: 5,
    3: 5,
    4: 1,
    5: 1,
    6: 1,
    7: 1,
    8: 1,
    9: 1
}
history_2, bird_cat_classifier_weights = compile_and_train(bird_cat_classifier, NUM_EPOCHS, train_images, train_labels, bird_cat_class_weight)
best_weight[1] = bird_cat_classifier_weights


In [ ]:
deer_dog_class_weight = {
    0: 1,
    1: 1,
    2: 1,
    3: 1,
    4: 5,
    5: 5,
    6: 1,
    7: 1,
    8: 1,
    9: 1
}
history_3, deer_dog_weights = compile_and_train(deer_dog_classifier, NUM_EPOCHS, train_images, train_labels, deer_dog_class_weight)
best_weight[2] = deer_dog_weights

In [ ]:
frog_horse_class_weight = {
    0: 1,
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 1,
    6: 5,
    7: 5,
    8: 1,
    9: 1
}
history_4, frog_horse_classifier_weights = compile_and_train(frog_horse_classifier, NUM_EPOCHS, train_images, train_labels, frog_horse_class_weight)
best_weight[3] = frog_horse_classifier_weights

In [ ]:
ship_truck_class_weight = {
    0: 1,
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 1,
    6: 1,
    7: 1,
    8: 5,
    9: 5
}
history_5, ship_truck_classifier_weights = compile_and_train(ship_truck_classifier, NUM_EPOCHS, train_images, train_labels, ship_truck_class_weight)
best_weight[4] = ship_truck_classifier_weights

# Voting-based Ensamble method

When making the voting based ensamble method we will initialize each classifier using the weights achieved in the training process, and then combining them using a weighted average.

In [6]:
model_input = Input(shape= train_images[0,:,:,:].shape)
models = [car_plane_classifier, bird_cat_classifier]

def ensemble(models: List [training.Model], model_input: Tensor) -> training.Model:
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    
    return model

In [7]:
best_weight = {0:"bird_and_cat.15-0.43.hdf5",1:"car_and_plane.15-0.31.hdf5",2:"deer_and_dog.15-0.35.hdf5",3:"frog_and_horse.15-0.30.hdf5",4:"ship_and_truck.15-0.32.hdf5"} 

In [9]:
CAR_PLANE_WEIGHT = os.path.join(os.getcwd(), '../weights/ensemble',best_weight[0] )
BIRD_CAT_WEIGHT = os.path.join(os.getcwd(), '../weights/ensemble', best_weight[1]  )
DEER_DOG_WEIGHT = os.path.join(os.getcwd(), '../weights/ensemble', best_weight[2] )
FROG_HORSE_WEIGHT = os.path.join(os.getcwd(), '../weights/ensemble', best_weight[3] )
SHIP_TRUCK_WEIGHT = os.path.join(os.getcwd(), '../weights/ensemble', best_weight[4] )

car_plane_classifier = cifar10_cnn_model(model_input, 'car_and_plane')
bird_cat_classifier = cifar10_cnn_model(model_input, 'bird_and_cat')
deer_dog_classifier = cifar10_cnn_model(model_input, 'deer_and_dog')
frog_horse_classifier = cifar10_cnn_model(model_input, 'frog_and_horse')
ship_truck_classifier = cifar10_cnn_model(model_input, 'ship_and_truck')

car_plane_classifier.load_weights(CAR_PLANE_WEIGHT)
bird_cat_classifier.load_weights(BIRD_CAT_WEIGHT)
deer_dog_classifier.load_weights(DEER_DOG_WEIGHT)
frog_horse_classifier.load_weights(FROG_HORSE_WEIGHT)
ship_truck_classifier.load_weights(SHIP_TRUCK_WEIGHT)

In [10]:
models = [car_plane_classifier, bird_cat_classifier, deer_dog_classifier, frog_horse_classifier, ship_truck_classifier]
ensemble_ = ensemble(models, model_input)

print(evaluate_error(ensemble_, test_images, test_labels))


0.1647
